<a href="https://colab.research.google.com/github/robsonspereira/COVID19/blob/master/corona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import plotly.express as px

In [0]:
url = "https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-2020-03-20.xlsx"
base_import = pd.read_excel(url)
base_import.head()
base_import = base_import[base_import['Cases']!=0]
base_import.dropna(inplace=True)
base_import = base_import.sort_values(by =['GeoId','DateRep'])
base_import.head()

,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId
165,2020-03-03,3,3,2020,1,0,Andorra,AD
164,2020-03-14,14,3,2020,1,0,Andorra,AD
163,2020-03-16,16,3,2020,3,0,Andorra,AD
162,2020-03-17,17,3,2020,9,0,Andorra,AD
160,2020-03-19,19,3,2020,39,0,Andorra,AD


In [0]:
Total_Cases = base_import[['GeoId','DateRep','Cases']]
Total_Cases['Total Cases']= Total_Cases.groupby(['GeoId']).cumsum()
Total_Cases.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,GeoId,DateRep,Cases,Total Cases
165,AD,2020-03-03,1,1
164,AD,2020-03-14,1,2
163,AD,2020-03-16,3,5
162,AD,2020-03-17,9,14
160,AD,2020-03-19,39,53


In [0]:
GeoIDGroup = base_import.groupby(['GeoId']).agg({'DateRep':['min','max'],'Cases':['sum'],'Deaths':['sum']})
GeoIDGroup.columns = ['DateFirstReport','DateLastReport','Cases','Deaths'] 
GeoIDGroup['Duration'] = (GeoIDGroup['DateLastReport']-GeoIDGroup['DateFirstReport'])
GeoIDGroup['Duration'] = pd.to_numeric(GeoIDGroup['Duration'].dt.days, downcast='integer')
GeoIDGroup['CasesPerDay'] = (GeoIDGroup['Cases']/GeoIDGroup['Duration'].astype(int))
GeoIDGroup['DeathsPerCases'] = (GeoIDGroup['Deaths']/GeoIDGroup['Cases'])
GeoIDGroup

,DateFirstReport,DateLastReport,Cases,Deaths,Duration,CasesPerDay,DeathsPerCases
GeoId,,,,,,,
AD,2020-03-03,2020-03-20,75,0,17,4.411765,0.000000
AE,2020-01-27,2020-03-20,140,0,53,2.641509,0.000000
AF,2020-02-25,2020-03-18,22,0,22,1.000000,0.000000
AG,2020-03-15,2020-03-15,1,0,0,inf,0.000000
AL,2020-03-09,2020-03-20,70,2,11,6.363636,0.028571
...,...,...,...,...,...,...,...
VE,2020-03-15,2020-03-17,33,0,2,16.500000,0.000000
VN,2020-01-24,2020-03-20,85,0,56,1.517857,0.000000
XK,2020-03-16,2020-03-20,21,0,4,5.250000,0.000000


In [0]:
GeoIDGroup.loc[['BR','IT','CN','US','ES','IE']]

,DateFirstReport,DateLastReport,Cases,Deaths,Duration,CasesPerDay,DeathsPerCases
GeoId,,,,,,,
BR,2020-02-26,2020-03-20,621,6,23,27.000000,0.009662
IT,2020-01-31,2020-03-20,41035,3407,49,837.448980,0.083027
CN,2019-12-31,2020-03-20,81337,3252,80,1016.712500,0.039982
US,2020-01-21,2020-03-20,14250,150,59,241.525424,0.010526
ES,2020-02-01,2020-03-20,17147,767,48,357.229167,0.044731
IE,2020-03-01,2020-03-20,557,3,19,29.315789,0.005386


In [0]:
  Events = base_import.merge(GeoIDGroup[['DateFirstReport']],left_on='GeoId',right_index=True, how ='left')
  Events['Day_report'] = (Events['DateRep']-Events['DateFirstReport']).dt.days.astype(int)
  Events= Events.merge(Total_Cases[['Total Cases']],left_index=True,right_index=True, how='left')
  Events

,DateRep,Day,Month,Year,Cases,Deaths,Countries and territories,GeoId,DateFirstReport,Day_report,Total Cases
165,2020-03-03,3,3,2020,1,0,Andorra,AD,2020-03-03,0,1
164,2020-03-14,14,3,2020,1,0,Andorra,AD,2020-03-03,11,2
163,2020-03-16,16,3,2020,3,0,Andorra,AD,2020-03-03,13,5
162,2020-03-17,17,3,2020,9,0,Andorra,AD,2020-03-03,14,14
160,2020-03-19,19,3,2020,39,0,Andorra,AD,2020-03-03,16,53
...,...,...,...,...,...,...,...,...,...,...,...
4896,2020-03-17,17,3,2020,11,0,South_Africa,ZA,2020-03-06,11,62
4895,2020-03-18,18,3,2020,23,0,South_Africa,ZA,2020-03-06,12,85
4894,2020-03-19,19,3,2020,31,0,South_Africa,ZA,2020-03-06,13,116
4893,2020-03-20,20,3,2020,34,0,South_Africa,ZA,2020-03-06,14,150


In [0]:
Events_filter=Events.query( 'Day_report<=30 ')
Events_filter= Events_filter.query('GeoId=="US" or GeoId=="IT" or GeoId=="CN" or GeoId=="BR"')

graph = px.line(Events_filter,
                x = 'Day_report',
                y = 'Cases',
                color = 'GeoId',
                line_group ='GeoId',
                hover_name = 'GeoId')

graph.show()

In [0]:
graph = px.line(Events_filter,
                x = 'Day_report',
                y = 'Total Cases',
                color = 'GeoId',
                line_group ='GeoId',
                hover_name = 'GeoId')

graph.show()